In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
DATA_DIR  = '/home/l-ashwin/Datasets/cifar-10-batches-py/'
UTILS_DIR = '/home/l-ashwin/Datasets/src_data/'

import sys
sys.path.append(UTILS_DIR)
from utils import load_cifar10

In [3]:
import numpy as np
import tensorflow as tf

# dataset

In [4]:
X_train, y_train, X_test, y_test = load_cifar10(DATA_DIR)

In [5]:
print(X_train.shape, y_train.shape)
print(X_test.shape,  y_test.shape )

(50000, 32, 32, 3) (50000,)
(10000, 32, 32, 3) (10000,)


In [6]:
train_data = tf.data.Dataset.from_tensor_slices((X_train.astype(np.float16), y_train))
test_data  = tf.data.Dataset.from_tensor_slices((X_test.astype(np.float16), y_test))

In [7]:
xx, yy = next(iter(train_data))
print(xx.shape, yy.shape)

(32, 32, 3) ()


In [8]:
BATCH_SIZE = 32

In [9]:
train_loader = train_data.batch(BATCH_SIZE)
test_loader  = test_data.batch(BATCH_SIZE)

In [10]:
xx, yy = next(iter(train_loader))
print(xx.shape, yy.shape)

(32, 32, 32, 3) (32,)


# model

In [11]:
base_model = tf.keras.applications.ResNet50(include_top=False, weights='imagenet')
base_model.trainable = False

In [12]:
Input  = tf.keras.Input(shape=xx.shape[1:])
x      = tf.keras.applications.vgg16.preprocess_input(Input)
x      = base_model(x)
x      = tf.keras.layers.Flatten()(x)
x      = tf.keras.layers.Dense(64, activation='relu')(x)
Output = tf.keras.layers.Dense(10)(x)

model  = tf.keras.Model(Input, Output)

In [13]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 32, 32, 3)         0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 32, 32, 3)         0         
_________________________________________________________________
resnet50 (Functional)        (None, None, None, 2048)  23587712  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                131136    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650   

# train

In [14]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss      = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics   = ['sparse_categorical_accuracy'])

In [15]:
model.fit(train_loader, validation_data=test_loader, epochs=5)

Epoch 1/5
1563/1563 [==============================] - 58s 36ms/step - loss: 1.4036 - sparse_categorical_accuracy: 0.5342 - val_loss: 1.0097 - val_sparse_categorical_accuracy: 0.6470
Epoch 2/5
1563/1563 [==============================] - 55s 35ms/step - loss: 0.9506 - sparse_categorical_accuracy: 0.6662 - val_loss: 0.9860 - val_sparse_categorical_accuracy: 0.6589
Epoch 3/5
1563/1563 [==============================] - 55s 35ms/step - loss: 0.8606 - sparse_categorical_accuracy: 0.6972 - val_loss: 0.9824 - val_sparse_categorical_accuracy: 0.6653
Epoch 4/5
1563/1563 [==============================] - 56s 36ms/step - loss: 0.7955 - sparse_categorical_accuracy: 0.7231 - val_loss: 0.9997 - val_sparse_categorical_accuracy: 0.6634
Epoch 5/5
1563/1563 [==============================] - 55s 35ms/step - loss: 0.7464 - sparse_categorical_accuracy: 0.7394 - val_loss: 1.0228 - val_sparse_categorical_accuracy: 0.6619
